In [1]:
%load_ext autoreload
%autoreload 2
import copy
from collections import OrderedDict, defaultdict

import numpy as np
import pandas as pd
from spikeextractors import RecordingExtractor, SortingExtractor
import spikeextractors as se

import spikemetrics.metrics as metrics
import spiketoolkit as st
from spikemetrics.utils import Epoch, printProgressBar
from spiketoolkit.curation.thresholdcurator import ThresholdCurator
import spiketoolkit as st

In [2]:
rec, sort = se.example_datasets.toy_example(duration=10, num_channels=4)

In [3]:
mc = st.validation.MetricCalculator(sort, rec)
mc.compute_all_metric_data(seed=0)

_ = mc.compute_metrics(seed=0)
metric_dict = mc.get_metrics_dict()
assert type(mc._recording._recording).__name__ == 'BandpassFilterRecording' #check if filter by default
assert 'firing_rate' in metric_dict.keys()
assert 'num_spikes' in metric_dict.keys()
assert 'isi_viol' in metric_dict.keys()
assert 'presence_ratio' in metric_dict.keys()
assert 'amplitude_cutoff' in metric_dict.keys()
assert 'max_drift' in metric_dict.keys()
assert 'cumulative_drift' in metric_dict.keys()
assert 'silhouette_score' in metric_dict.keys()
assert 'isolation_distance' in metric_dict.keys()
assert 'l_ratio' in metric_dict.keys()
assert 'd_prime' in metric_dict.keys()
assert 'nn_hit_rate' in metric_dict.keys()
assert 'nn_miss_rate' in metric_dict.keys()
assert 'snr' in metric_dict.keys()
assert mc.is_filtered()

In [4]:
from spiketoolkit.validation import AmplitudeCutoff
from spiketoolkit.validation import SilhouetteScore
from spiketoolkit.validation import MetricData
from spiketoolkit.validation.quality_metrics_new import compute_amplitude_cutoffs
from spiketoolkit.validation.quality_metrics_new import compute_silhouette_scores

from spiketoolkit.validation import NumSpikes
from spiketoolkit.validation.quality_metrics_new import compute_num_spikes

from spiketoolkit.validation import DPrime
from spiketoolkit.validation.quality_metrics_new import compute_d_primes

In [5]:
#num spikes
print("num spikes")
md = MetricData(sorting=sort, sampling_frequency=rec.get_sampling_frequency(), apply_filter=True)
ns = NumSpikes(metric_data=md)
ns_metric = compute_num_spikes(sorting=sort, sampling_frequency=rec.get_sampling_frequency())
print(np.equal(ns.compute_metric(), metric_dict['num_spikes']))
print(np.equal(ns_metric, metric_dict['num_spikes']))

#amp cutoff
print("amp cutoff")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_amplitudes(seed=0)
ac = AmplitudeCutoff(metric_data=md)
ac_metric = compute_amplitude_cutoffs(sorting=sort, recording=rec, seed=0)
print(np.equal(ac.compute_metric(), metric_dict['amplitude_cutoff']))
print(np.equal(ac_metric, metric_dict['amplitude_cutoff']))

#silhouette score
print("silhouette score")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_pca_scores(seed=0)
ss = SilhouetteScore(metric_data=md)
ss_metric = compute_silhouette_scores(sorting=sort, recording=rec, seed=0)
print(np.equal(ss.compute_metric(seed=0), metric_dict['silhouette_score']))
print(np.equal(ss_metric, metric_dict['silhouette_score']))

#d primes
print("d primes")
md = MetricData(sorting=sort, recording=rec, apply_filter=True)
md.compute_pca_scores(seed=0)
dp = DPrime(metric_data=md)
dp_metric = compute_d_primes(sorting=sort, recording=rec, seed=0)
print(np.equal(dp.compute_metric(seed=0), metric_dict['d_prime']))
print(np.equal(dp_metric, metric_dict['d_prime']))

num spikes
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
amp cutoff
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
silhouette score
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]
d primes
[[ True  True  True  True  True  True  True  True  True  True]]
[[ True  True  True  True  True  True  True  True  True  True]]


In [6]:
from spiketoolkit.curation.threshold_metrics import threshold_amplitude_cutoffs
from spiketoolkit.curation.threshold_metrics import threshold_silhouette_scores
from spiketoolkit.curation.threshold_metrics import threshold_num_spikes
from spiketoolkit.curation.threshold_metrics import threshold_d_primes

In [12]:
#amp cutoff
print("amp cutoff")
sorting_new = threshold_amplitude_cutoffs(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater')
sorting_old = st.curation.threshold_amplitude_cutoff(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater')
print(compute_amplitude_cutoffs(sorting=sorting_new, recording=rec))
print(np.equal(compute_amplitude_cutoffs(sorting=sorting_new, recording=rec), compute_amplitude_cutoffs(sorting=sorting_old, recording=rec)))

#silhouette scores
print("silhouette scores")
sorting_new = threshold_silhouette_scores(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater', seed=0)
sorting_old = st.curation.threshold_silhouette_score(sorting=sort, recording=rec, threshold=.4, threshold_sign='greater', seed=0)
print(compute_silhouette_scores(sorting=sorting_new, recording=rec, seed=0))
print(np.equal(compute_silhouette_scores(sorting=sorting_new, recording=rec, seed=0), compute_silhouette_scores(sorting=sorting_old, recording=rec, seed=0)))

#amp cutoff
print("num spikes")
sorting_new = threshold_num_spikes(sorting=sort, threshold=24, threshold_sign='less')
sorting_old = st.curation.threshold_num_spikes(sorting=sort, threshold=24, threshold_sign='less')
print(compute_num_spikes(sorting=sorting_new))
print(np.equal(compute_num_spikes(sorting=sorting_new), compute_num_spikes(sorting=sorting_old)))

#d primes
print("d primes")
sorting_new = threshold_d_primes(sorting=sort, recording=rec, threshold=5, threshold_sign='greater', seed=0)
sorting_old = st.curation.threshold_d_primes(sorting=sort, recording=rec, threshold=5, threshold_sign='greater', seed=0)
print(compute_d_primes(sorting=sorting_new, recording=rec, seed=0))
print(np.equal(compute_d_primes(sorting=sorting_new, recording=rec, seed=0), compute_d_primes(sorting=sorting_old, recording=rec, seed=0)))

amp cutoff
[array([0.01078261, 0.01444225, 0.15770606, 0.0103513 , 0.0103513 ,
       0.03234782, 0.33842186, 0.1703513 , 0.01078446])]
[[ True  True  True  True  True  True  True  True  True]]
silhouette scores
[array([0.32017871, 0.32017871, 0.06255415, 0.11224128, 0.06255415,
       0.194613  ])]
[[ True  True  True  True  True  True]]
num spikes
[array([24., 24., 27., 25., 25., 24., 24., 25., 24.])]
[[ True  True  True  True  True  True  True  True  True]]
d primes
[array([10.83804245,  1.75973176,  4.17817877,  6.65368591,  2.9565444 ,
        2.84865292,  8.88413306])]
[[ True  True  True  True  True  True  True]]
